In [1]:
import pandas as pd
import pathlib
import seaborn as sns

from matplotlib import pyplot as plt
from pprint import pprint

from settings import (BASE_CSV_PATH, QUALTRICS_DROP_COLS, SKIP_COLS, 
                      MULTISELECT_COLS, AGREE_DISAGREE_COLS, IMPORTANCE_COLS, 
                      STANDARD_DUMMY_COLS, UTILITY_SCORE_COLS)

In [2]:
base_df = pd.read_csv(BASE_CSV_PATH)
base_df.shape

(675, 139)

Only 675 responses, I assume?

## Pre-processing functions

In [3]:
def drop_utility_question_columns(in_df):
    out_df = in_df.copy()
    util_cols = [c for c in in_df.columns if "(This leaves 1 choice" in c]
    return out_df.drop(labels = util_cols, axis = 1)

In [4]:
def create_multinational_column(in_df, op_column = "Where does your organization have offices", 
                                null_fill = ""):
    out_df = in_df.copy()
    out_df["is_multinational"] = out_df[op_column].fillna(null_fill)
    out_df["is_multinational"] = out_df.is_multinational.str.contains("Multi")
    out_df["is_multinational"] = out_df.is_multinational.astype(int)
    return out_df.drop(labels = op_column, axis = 1)

In [5]:
def create_executive_column(in_df, op_column = "What is your seniority level?", 
                                null_fill = ""):
    out_df = in_df.copy()
    out_df["is_executive"] = out_df[op_column].fillna(null_fill)
    out_df["is_executive"] = out_df.is_executive.str.contains("Exec")
    out_df["is_executive"] = out_df.is_executive.astype(int)
    return out_df.drop(labels = op_column, axis = 1)

In [6]:
def create_male_column(in_df, op_column = "What best describes your gender?", 
                                null_fill = ""):
    out_df = in_df.copy()
    out_df["is_male"] = out_df[op_column].fillna(null_fill)
    out_df["is_male"] = out_df.is_male.str.contains("Male")
    out_df["is_male"] = out_df.is_male.astype(int)
    return out_df.drop(labels = op_column, axis = 1)

In [7]:
def add_cluster_columns(in_df, op_column = "labels"):
    out_df = in_df.copy()
    out_df["clust_zeroAndOne"] = [1 if v in [0,1] else 0 for v in in_df[op_column]]
    out_df["clust_eight"] = [1 if v == 8 else 0 for v in in_df[op_column]]
    out_df["clust_sixAndNine"] = [1 if v in [6,9] else 0 for v in in_df[op_column]]
    return out_df.drop(labels = op_column, axis = 1)

In [8]:
def add_dummmies_to_df(in_df, col_to_code, coding_prefix, prefix_sep = "_"):
    ''' Will work for naively dropping levels where there's no specific meaning to
    the effect and it's just belonging to a category
    
    I may need to handle the 'agree - disagree' columns separately...
    '''
    level_count = in_df[col_to_code].nunique()
    if level_count < 3:
        print(f"\tColumn {col_to_code} only has {level_count} levels...")
        print(f"\tReturning unmodified df")
        return in_df
    dummy_df = pd.get_dummies(data = in_df[col_to_code], 
                              prefix = coding_prefix, 
                              prefix_sep = prefix_sep)
    dummy_cols = dummy_df.columns.tolist()
    drop_col_index = dummy_df.sum().argmin()
    drop_col = dummy_cols[drop_col_index]
    print(f"\tRemoving level {drop_col} from coding for {col_to_code}")
    dummy_df.drop(labels = drop_col, axis = 1, inplace = True)
    out_df = in_df.join(dummy_df, how = "inner")
    return out_df.drop(labels = col_to_code, axis = 1)

## Preprocessing Steps

**Remove columns that are component ingredients for the overall utility scores:** Any column with `(This leaves 1 choice` can be dropped off, just because these are the component ingredients for the overall utilities.

In [9]:
reg_df = drop_utility_question_columns(in_df = base_df)

In [10]:
reg_df.shape

(675, 96)

**Remove columns that you haven't figured out how to fully deal with yet**

In [11]:
for column_group in [QUALTRICS_DROP_COLS, SKIP_COLS, MULTISELECT_COLS, AGREE_DISAGREE_COLS, IMPORTANCE_COLS]:
    reg_df.drop(labels = column_group, axis = 1, inplace = True)

In [12]:
reg_df.shape

(675, 27)

**Use the functions above to create special columns for multinational, executive, and gender identity of response...**

In [13]:
reg_df = create_multinational_column(in_df = reg_df)
reg_df = create_executive_column(in_df = reg_df)
reg_df = create_male_column(in_df = reg_df)

In [14]:
reg_df.shape

(675, 27)

**Create dummy columns in your standardized way, only keeping an effect for the least frequent response of each type**

In [15]:
for code_column, prefix in STANDARD_DUMMY_COLS.items():
    reg_df = add_dummmies_to_df(in_df = reg_df, 
                                col_to_code = code_column,
                                coding_prefix = prefix)

	Removing level empFunc_Design from coding for Which business function best describes your core responsibility?
	Removing level bizSize_Sole proprietor from coding for What best describes your company size?
	Removing level buildCapability_I’m responsible for training people on a new software from coding for What best describes your role in building capabilities within your organization?  Select the most relevant.
	Removing level bizStrategy_Other from coding for Which of the following statements best describes your company’s strategy today?
	Removing level budgetCategory_Other from coding for What type of budget might you use for a solution similar to that of the concept "Learn While Doing?" - Selected Choice
	Removing level purchaseRole_I’d approve the purchase for my entire organization from coding for What would best describe your involvement in the purchase process of a solution similar to that of the concept "Learn While Doing?"
	Removing level empAge_18-21 from coding for What is

In [16]:
reg_df.shape

(675, 80)

**Create custom columns for the clusters that Nathan and Isabella specified:** this uses and then gets rid of the `label` column

In [17]:
reg_df = add_cluster_columns(in_df = reg_df)

In [18]:
reg_df.shape

(675, 82)

**Finally, drop out people with a `fit_statistic` lower than 0.39:** Apparently this is what sawtooth recommends when looking at utilities. I also think that Alla may have already taken this step when creating this file...

In [19]:
reg_df = reg_df.loc[(reg_df["Fit Statistic"] > 0.39)]
reg_df.drop(labels = "Fit Statistic", axis = 1, inplace = True)
reg_df.shape

(675, 81)

**OK,** so the only extraneous columns at this point are the value prop columns related to the target.

In [20]:
pprint(reg_df.columns.tolist())

["Equip your workforce with modern skills while solving your organization's "
 'top strategic challenges',
 'Build new capabilities while doing the work that matters to your team',
 "Hone your team's ability to continually experiment and build on the "
 'resulting knowledge',
 'Equip teams with the tools and mindset to solve problems independently',
 'Stay ahead of industry disruption by fostering an agile, resilient '
 'organizational culture',
 'Lead your organization through a digital transformation',
 'Cultivate shared processes and mindsets so your team can achieve better '
 'results',
 'Develop customer-centered skills to repeatedly build products, services, and '
 'experiences that appeal to your target audience',
 'Help employees develop customer-centered skills while working towards '
 'company strategy',
 'Scale new processes and methodologies across your organization with tools '
 'every team will find valuable',
 'is_multinational',
 'is_executive',
 'is_male',
 'empFunc_En

## Making a Target Variable

The only other features we may want to create are **which** of the 10 value propositions the person rated _most_ or _least_ desireable..

And which of those columns we want to keep depends on whether we're regressing against max utility or min utility.

In [21]:
max_utility_scores = reg_df[UTILITY_SCORE_COLS].max(axis = 1)
max_utility_scores.shape

(675,)

This is the target variable for the maximum utility score...

In [22]:
max_utility_scores.describe()

count    675.000000
mean      20.696342
std        1.873814
min       14.738210
25%       19.558480
50%       20.758560
75%       22.047345
max       25.626480
dtype: float64

In [23]:
util_score_df = reg_df[UTILITY_SCORE_COLS].copy()

In [24]:
max_utility_vps = util_score_df.idxmax(axis = 1)
max_utility_df = pd.DataFrame(data = max_utility_vps, columns = ["maxUtility"])
max_utility_df

,maxUtility
0,Help employees develop customer-centered skill...
1,Equip teams with the tools and mindset to solv...
2,Develop customer-centered skills to repeatedly...
3,Equip teams with the tools and mindset to solv...
4,Cultivate shared processes and mindsets so you...
...,...
670,Hone your team's ability to continually experi...
671,Scale new processes and methodologies across y...
672,Develop customer-centered skills to repeatedly...
673,Develop customer-centered skills to repeatedly...


In [25]:
max_utility_df["maxUtility"].value_counts()

Develop customer-centered skills to repeatedly build products, services, and experiences that appeal to your target audience    124
Lead your organization through a digital transformation                                                                         119
Stay ahead of industry disruption by fostering an agile, resilient organizational culture                                        78
Hone your team's ability to continually experiment and build on the resulting knowledge                                          63
Help employees develop customer-centered skills while working towards company strategy                                           58
Equip teams with the tools and mindset to solve problems independently                                                           57
Equip your workforce with modern skills while solving your organization's top strategic challenges                               46
Scale new processes and methodologies across your organization with tools ev

In [26]:
max_utility_feats = add_dummmies_to_df(in_df = max_utility_df, col_to_code="maxUtility", coding_prefix="maxUtil")

	Removing level maxUtil_Build new capabilities while doing the work that matters to your team from coding for maxUtility


In [27]:
max_utility_feats.shape

(675, 9)

In [28]:
reg_df.shape

(675, 81)

In [29]:
len(UTILITY_SCORE_COLS)

10

So, the resulting `max_utility_feats` df should be 81 - 10 + 9 columns wide, or 80.

In [30]:
base_feat_cols = [c for c in reg_df.columns if c not in UTILITY_SCORE_COLS]
max_utility_feats = max_utility_feats.join(reg_df[base_feat_cols], how = "inner")

In [31]:
max_utility_feats.shape

(675, 80)

Basically repeating the process above with the minimum percentile score (... more for formality's sake ...)

In [38]:
min_utility_scores = reg_df[UTILITY_SCORE_COLS].min(axis = 1)
min_utility_scores.shape

(675,)

In [39]:
min_utility_vps = util_score_df.idxmin(axis = 1)
min_utility_df = pd.DataFrame(data = min_utility_vps, columns = ["minUtility"])
min_utility_feats = add_dummmies_to_df(in_df = min_utility_df, col_to_code="minUtility", coding_prefix="minUtil")
min_utility_feats = min_utility_feats.join(reg_df[base_feat_cols], how = "inner")

	Removing level minUtil_Scale new processes and methodologies across your organization with tools every team will find valuable from coding for minUtility


In [42]:
min_utility_feats.shape

(675, 80)

In [41]:
min_utility_df["minUtility"].value_counts()

Lead your organization through a digital transformation                                                                         180
Hone your team's ability to continually experiment and build on the resulting knowledge                                         104
Stay ahead of industry disruption by fostering an agile, resilient organizational culture                                        68
Help employees develop customer-centered skills while working towards company strategy                                           65
Cultivate shared processes and mindsets so your team can achieve better results                                                  60
Develop customer-centered skills to repeatedly build products, services, and experiences that appeal to your target audience     54
Build new capabilities while doing the work that matters to your team                                                            39
Equip your workforce with modern skills while solving your organization's to

## Trying a regression...

In [32]:
import statsmodels.api as sm

Adding a bias term here seems like a good idea...

In [34]:
max_utility_feats = sm.add_constant(max_utility_feats)

In [35]:
model = sm.OLS(max_utility_scores, max_utility_feats)

In [36]:
results = model.fit()

In [37]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.309
Model:                            OLS   Adj. R-squared:                  0.216
Method:                 Least Squares   F-statistic:                     3.323
Date:                Wed, 01 Jul 2020   Prob (F-statistic):           6.05e-17
Time:                        17:27:14   Log-Likelihood:                -1256.3
No. Observations:                 675   AIC:                             2675.
Df Residuals:                     594   BIC:                             3040.
Df Model:                          80                                         
Covariance Type:            nonrobust                                         
                                                                                                                                           coef    std err          t      P>|t|      [0.025      0.975]
---------

Accidentally running this without the bias term added yielded very different results(but admittedly less believable). It may be more useful in this case, but I would need to change variables a bit to **not** drop the k-1 variables. See [these nerds on cross validated](https://stats.stackexchange.com/questions/7948/when-is-it-ok-to-remove-the-intercept-in-a-linear-regression-model#:~:text=If%20your%20response%20variable%20is,to%20leave%20the%20intercept%20in.&text=You%20can%20leave%20out%20the,or%20your%20residuals%20are%20biased.) for more details.

**TODO (?)**: experiment with this, because I don't think it would change most of your processing functions _that much?_

In [43]:
min_utility_feats = sm.add_constant(min_utility_feats)

model = sm.OLS(max_utility_scores, min_utility_feats)

results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.130
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.110
Date:                Wed, 01 Jul 2020   Prob (F-statistic):              0.252
Time:                        18:20:04   Log-Likelihood:                -1334.1
No. Observations:                 675   AIC:                             2830.
Df Residuals:                     594   BIC:                             3196.
Df Model:                          80                                         
Covariance Type:            nonrobust                                         
                                                                                                                                           coef    std err          t      P>|t|      [0.025      0.975]
---------